<a href="https://colab.research.google.com/github/AdamCorbinFAUPhD/CIRCLe-experiments/blob/main/densenet121/2023_01_04/CIRCLe_with_isic2018_with_skin_transformer_densenet121_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

This notebook is used to modify the implementation of CIRCLe from this paper : [CIRCLe: Color Invariant Representation
Learning for Unbiased Classification of Skin
Lesions](https://arxiv.org/pdf/2208.13528.pdf)

Their github repo is : https://github.com/arezou-pakzad/CIRCLe

This paper uses the Fitzpatrick17k dataset which can be obtained here: https://github.com/mattgroh/fitzpatrick17k

For these set of experiments we will use the ISIC 2017 dataset from: https://github.com/manideep2510/melanoma_segmentation.git 

#TODO list

1. [X] Download 2018 dataset
1. [X] Analize dataset to get Fitzpatrick info. 
1. [X] Save off Fitzpatrick info data so we dont have to do it every time
1. [X] load cached fitzpatrick data
1. [X] Create masks uing https://github.com/DebeshJha/2020-CBMS-DoubleU-Net Because Task 3 for 2018 doesnt havent masks. Trick was to get the higher end GPU and ram (12/29/2022)
1. [X] Create pytorch dataloader for ISIC 2018 dataset including loading masks, images, diagnossis, fitzpatrick type for training (12/30/2022) needed to create custom split function
1. [X] Create dataloaders for test and validation  (12/30/2022)
1. [X] Added jupiter notebook download code into the github repo (1/1/2023)
1. [X] plug in dataloader into CIRCLe main file (1/1/2023)
1. [X] Figure out how to transform image and mask the same from the dataloader (1/2/2023)
1. [X] Use the new dataloader to train the model (1/2/2023)
1. [X] Use new transformer for CIRCLe model (1/3/2023)
1. [ ] test using different base models
1. [ ] test that adding dropout might help with overfitting
1. [ ] Add more metrics such as precision and recall
1. [ ] add fairness metrics
1. [ ] add confusion matrics
1. [ ] add sensitivity and specificity
1. [ ] add metrics for each class
1. [ ] Turn back on the random rotation and flipping in the dataloader
1. [ ] (optional) Go back and download and use larger datasets
1. [ ] (optional) Run Fitzpatrick on larger datasets(currently using the test set from isic 2018 task 3)
1. [ ] The dataloaders need to be split stratified different than the current "training, validation, and test" as given from https://challenge.isic-archive.com/data/#2018 based on skin types. 12/30/2022 - I think this is done BUT we might consider doing k-fold approach which adds another layer of complexity to the dataloaders

# Set up the environment

In [1]:
!python --version
DATASET_USED = "ISIC_2018"  # ISIC_2017_ORIG, ISIC_2018

Python 3.8.16


## Installs & imports

In [2]:
import zipfile
import torch

from enum import Enum
import io
import math
from pathlib import Path
import random

import numpy as np
import seaborn as sns, matplotlib.pyplot as plt


import skimage
from skimage import color, util

import PIL
from PIL import ImageStat
from PIL import Image
from PIL import ImageOps

import cv2

import pandas as pd

import glob
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.metrics import *

from sklearn.model_selection import train_test_split

from torchvision import transforms

## Download latest code

In [3]:
!git clone https://github.com/acorbin3/CIRCLe.git

fatal: destination path 'CIRCLe' already exists and is not an empty directory.


In [4]:
%cd ./CIRCLe

/content/CIRCLe


In [5]:
!git checkout -- models/circle.py

In [6]:
!git pull

Already up to date.


In [7]:
!pip3 install -r ./requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# IF ERROR, RESTART RUNTIME due to derm-ita lib
This is due to derm-ita using newer libaries than the Google Colab default(during this time of 12/24/2022)

# Train CIRCLe model 

In [8]:
%mkdir ./saved
%mkdir ./saved/model

mkdir: cannot create directory ‘./saved’: File exists
mkdir: cannot create directory ‘./saved/model’: File exists


In [9]:
!git checkout -- ./models/circle.py

In [10]:
!git checkout -- main.py

In [11]:
!git pull

Already up to date.


In [ ]:
!python main.py --use_reg_loss True --base densenet121 --dataset isic2018

Flags:
	alpha: 0.1
	base: densenet121
	batch_size: 32
	data_dir: ../data/fitz17k/images/all/
	dataset: isic2018
	epochs: 100
	gan_path: saved/stargan/
	hidden_dim: 256
	lr: 0.001
	model: circle
	model_save_dir: saved/model/
	num_classes: 7
	seed: 1
	use_reg_loss: True
	weight_decay: 0.001
tcmalloc: large alloc 2771738624 bytes == 0xce970000 @  0x7f6e7ee251e7 0x4d30a0 0x5dede2 0x6758aa 0x4f750a 0x4997a2 0x4f700d 0x4d4aa9 0x55e029 0x55cd91 0x5d8941 0x4fe318 0x5d8416 0x55f797 0x55cd91 0x5d8941 0x4fe318 0x5d8416 0x55f797 0x55cd91 0x5d8941 0x5d8416 0x55f797 0x55cd91 0x5d8941 0x4997c7 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4990ca
Resizing masks
Resizing masks. Complete!
Donloading isic 2018 ground truth classification data
Creating dataframe
	 Looking for cached dataframe
		 organize_data/isic_2018/saved_data_2022_12_27_isic_2018.csv
Creating dataframe. Complete!
Splitting up the dataset into train,test, validation datasets
fizpatrick_skin_type: 1 8001
	 train 6400
	 test 800
	 val 801
fizpat

In [ ]:
#%mkdir /content/drive/MyDrive/Corbin_Adam_PhD_Workspace/corbin_papers/dissertation_proposal/model_checkpoints

In [ ]:
#%cp ./saved/model/epoch97_acc_0.762.ckpt /content/drive/MyDrive/Corbin_Adam_PhD_Workspace/corbin_papers/dissertation_proposal/model_checkpoints/CIRCLE/mobilenetv3l/